In [1]:

print('start')

start


In [2]:
import glob
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import re
from DataProcessing import *
from DataImporting import sequence_list
peak_list, residue_list = main_data_processing()

ModuleNotFoundError: No module named 'pandas'

In [14]:
two_d_dict={}
two_d_dict['HShift'] = [x.get_data('TROSYHShift') for x in peak_list]
two_d_dict['NShift'] = [x.get_data('TROSYNShift') for x in peak_list]
two_d_dict['CA'] = [x.get_data('CAShift') for x in peak_list]
two_d_dict['CB'] = [x.get_data('CBShift') for x in peak_list]
two_d_dict['CAPrime'] = [x.get_data('CAPrimeShift') for x in peak_list]
two_d_dict['CBPrime'] = [x.get_data('CBPrimeShift') for x in peak_list]
two_d_dataframe = pd.DataFrame(two_d_dict)

closest_ca_list, closest_cb_list = [], []
for row1 in range(len(two_d_dataframe)):
    ca_shift = two_d_dataframe.loc[row1, "CA"]
    cb_shift = two_d_dataframe.loc[row1, "CB"]
    if np.isnan(ca_shift):
        closest_ca_list.append(None)
    else:
        ls_ca = []
        for row2 in range(len(two_d_dataframe)):
            ca_prime_shift = two_d_dataframe.loc[row2, "CAPrime"]
            if not np.isnan(ca_prime_shift):
                ls_ca.append(round(abs(ca_shift - ca_prime_shift), 4))
        ls_ca.sort()
        closest_ca_list.append(ls_ca[0:5])

    if np.isnan(cb_shift):
        closest_cb_list.append(None)
    else:
        ls_cb = []
        for row2 in range(len(two_d_dataframe)):
            cb_prime_shift = two_d_dataframe.loc[row2, "CBPrime"]
            if not np.isnan(cb_prime_shift):
                ls_cb.append(round(abs(cb_shift - cb_prime_shift), 4))
        ls_cb.sort()
        closest_cb_list.append(ls_cb[0:5])

two_d_dataframe['ClosestCAShift1D'] = closest_ca_list
two_d_dataframe['ClosestCBShift1D'] = closest_cb_list

closest_2d_list = []
row_number = []
ca_del_shift_list = []
cb_del_shift_list = []
for row1 in range(len(two_d_dataframe)):
    ca_shift = two_d_dataframe.loc[row1, "CA"]
    cb_shift = two_d_dataframe.loc[row1, "CB"]
    if np.isnan(ca_shift):
        closest_2d_list.append(None)
        row_number.append(None)
        ca_del_shift_list.append(None)
        cb_del_shift_list.append(None)
    elif np.isnan(cb_shift):
        closest_2d_list.append(None)
        row_number.append(None)
        ca_del_shift_list.append(None)
        cb_del_shift_list.append(None)
    else:
        ls = []
        ls1 = []
        ls_ca, ls_cb = [], []
        for row2 in range(len(two_d_dataframe)):
            ca_prime_shift = two_d_dataframe.loc[row2, "CAPrime"]
            cb_prime_shift = two_d_dataframe.loc[row2, "CBPrime"]
            if not np.isnan(ca_prime_shift) and not np.isnan(cb_prime_shift):
                ca_delta_squared = (abs(ca_shift - ca_prime_shift)) ** 2
                cb_delta_squared = (abs(cb_shift - cb_prime_shift)) ** 2
                ls.append(((ca_delta_squared + cb_delta_squared)/2) ** .5)
                ls1.append(row2)
                ls_ca.append(abs(ca_shift - ca_prime_shift))
                ls_cb.append(abs(cb_shift - cb_prime_shift))
        ls, ls1, ls_ca, ls_cb = zip(*sorted(zip(ls, ls1, ls_ca, ls_cb)))
        closest_2d_list.append([round(x, 5) for x in ls[0:5]])
        row_number.append(ls1[0:5])
        ca_del_shift_list.append([round(x, 5) for x in ls_ca[0:5]])
        cb_del_shift_list.append([round(x, 5) for x in ls_cb[0:5]])
        
two_d_dataframe['Closest2DShift (ppm)'] = closest_2d_list
two_d_dataframe['Corresponding peak'] = row_number
two_d_dataframe['CADel'] = ca_del_shift_list
two_d_dataframe['CBDel'] = cb_del_shift_list

In [15]:
print(two_d_dataframe.head())

   HShift   NShift      CA      CB  CAPrime  CBPrime  \
0   5.756  114.300  58.590  57.296   60.265      NaN   
1   5.251  126.472  55.967     NaN      NaN      NaN   
2   6.303  122.611  49.834  69.595   49.177   66.158   
3   6.425  120.326  56.740  55.626   58.271   22.790   
4   6.285  109.235  63.646  56.981   63.742   57.774   

                      ClosestCAShift1D                     ClosestCBShift1D  \
0  [0.089, 0.134, 0.178, 0.229, 0.231]    [0.014, 0.022, 0.05, 0.058, 0.18]   
1  [0.018, 0.053, 0.067, 0.081, 0.089]                                 None   
2    [0.09, 0.105, 0.33, 0.657, 0.884]  [0.307, 0.557, 0.587, 0.614, 0.775]   
3  [0.024, 0.041, 0.089, 0.164, 0.173]  [0.018, 0.021, 0.024, 0.087, 0.151]   
4    [0.014, 0.035, 0.04, 0.07, 0.075]  [0.068, 0.073, 0.102, 0.122, 0.128]   

                            Closest2DShift (ppm)        Corresponding peak  \
0   [0.14644, 0.17141, 0.29855, 0.52331, 0.5289]   (180, 101, 11, 80, 331)   
1                               

In [23]:
twod_delta_list_zero = [x[0] for x in two_d_dataframe['Closest2DShift (ppm)'] if x is not None and x[0] < 5]
twod_delta_list_one = [x[1] for x in two_d_dataframe['Closest2DShift (ppm)'] if x is not None and x[1] < 5]
plt.hist(twod_delta_list_zero, color='b', bins=20)
plt.xticks(np.arange(0, 5, step=.5))
plt.yticks(np.arange(0, 50, step=5))
plt.show()
plt.hist(twod_delta_list_one, color='r', bins=20)
plt.xticks(np.arange(0, 5, step=.5))
plt.yticks(np.arange(0, 50, step=5))
plt.show()

In [5]:
import plotly.express as px
import plotly.graph_objects as go 

In [52]:
twod_delta_list_zero = [x[0] for x in two_d_dataframe['Closest2DShift (ppm)'] if x is not None and x[0] < 5]
twod_delta_list_one = [x[1] for x in two_d_dataframe['Closest2DShift (ppm)'] if x is not None and x[1] < 5]
fig0 = go.Figure()
fig0.add_trace(go.Histogram(x = twod_delta_list_one, name='second closest', opacity = .75))
fig0.add_trace(go.Histogram(x = twod_delta_list_zero, name='first closest', opacity = .75))

fig0.update_layout(barmode = 'overlay', title_text='Closest 2D Shift Histogram', 
                  xaxis_title_text='Root Mean Square Delta Shift (ppm)')
fig0.show()

In [55]:
x0 = [x[0] for x in two_d_dataframe['CADel'] if x is not None and x[0] < 2]
y0 = [y[0] for y in two_d_dataframe['CBDel'] if y is not None and y[0] < 2]
x1 = [x[1] for x in two_d_dataframe['CADel'] if x is not None and x[1] < 2]
y1 = [y[1] for y in two_d_dataframe['CBDel'] if y is not None and y[1] < 2]

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=x1, y=y1, name='second closest', mode='markers', opacity = .5))
fig1.add_trace(go.Scatter(x=x0, y=y0, name='first closest', mode='markers'))
fig1.update_layout(yaxis_title_text='CBDel', yaxis=dict(scaleanchor = 'x', scaleratio=1), 
                  xaxis_title_text='CADel', title_text = 'Closest 2D Shift Scatterplot (2ppm truncated)',
                  width=800, height=800)
fig1.show()

In [53]:
import chart_studio.plotly as py
chart_studio.tools.set_credentials_file(username='sk1617', api_key='htVpHm0fYKP0sT2H5vHu')

py.iplot(fig0)


In [56]:
py.iplot(fig1)